----
# **0. Install Library**
----

In [4]:
# 라이브러리 설치
!pip install tqdm
!pip install konlpy

# konlpy, Mecab 설치
# Reference: https://teddylee777.github.io/colab/colab-mecab
!curl -s https://raw.githubusercontent.com/teddylee777/machine-learning/master/99-Misc/01-Colab/mecab-colab.sh | bash

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 19.4 MB 17.2 MB/s 
     |████████████████████████████████| 453 kB 31.3 MB/s 
--2022-06-08 07:39:07--  https://www.dropbox.com/s/9xls0tgtf3edgns/mecab-0.996-ko-0.9.2.tar.gz?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.85.18, 2620:100:6017:18::a27d:212
Connecting to www.dropbox.com (www.dropbox.com)|162.125.85.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/dl/9xls0tgtf3edgns/mecab-0.996-ko-0.9.2.tar.gz [following]
--2022-06-08 07:39:08--  https://www.dropbox.com/s/dl/9xls0tgtf3edgns/mecab-0.996-ko-0.9.2.tar.gz
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc04596281416be3dac6c19d001c.dl.dropboxusercontent.com/cd/0/get

----
# **1. Data Cleaning**
----
 - 결측치 확인 => 제거
 - 중복 리뷰 확인 => 제거

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

In [2]:
genre_list = ['다큐멘터리', '뮤지컬', '전쟁', '액션', '공포', '느와르', '모험', 'SF', '스릴러', '범죄', '멜로애정로맨스', '판타지', '코미디', '애니메이션', '미스터리', '가족', '드라마']

In [3]:
# 데이터 로드
genre = '드라마'
PATH = '/content/drive/MyDrive/final-project/data'
df = pd.read_csv(PATH + f'/movie_crawling_{genre}.csv', index_col=0)
df.head()

,genre,movie_name,score,review
0,드라마,그린 북,10,폭설내리는밤 경찰이 두번째로 주인공이 몰고가는 차를 세웠을때 우린 경찰의 대사나 행...
1,드라마,그린 북,10,우리가 말모이와 스윙키즈처럼 중요한 역사적 소재에 빚지고도 허구의 신파 한가득 버무...
2,드라마,그린 북,10,"하.. 이런 영화가 상영관수, 상영시간을 많이 늘려야 되는데.. 진짜 예고편만 화려..."
3,드라마,그린 북,10,보고나면 KFC 치킨이 존나 떙기는 영화임..
4,드라마,그린 북,10,빗속 셜리의 대사는 소름이 끼쳤다


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 816159 entries, 0 to 816158
Data columns (total 4 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   genre       816159 non-null  object
 1   movie_name  816159 non-null  object
 2   score       816159 non-null  int64 
 3   review      805090 non-null  object
dtypes: int64(1), object(3)
memory usage: 31.1+ MB


In [6]:
# NaN, 관람객 같은 리뷰 존재 => 데이터 클리닝 필요
# "배우"들의 "연기"와 "시각", "음향"이 좋았다. "시간"의 "비선형성"이라는 "소재"로 ...
# => 키워드 추출
df.sample(10)

,genre,movie_name,score,review
281575,드라마,우리집,8,관람객
322945,드라마,오두막,10,너무나 감동적인 강추하고픈 영화입니다.
20277,드라마,굿바이 마이 프랜드,10,진짜진짜 친구란좋은것인걸알았어요..
267545,드라마,채비,9,영화도 담백하게 잘 봤습니당ㅎㅎ 그리고 김성균씨 연기 최곱니당ㅠㅠ!!
501262,드라마,녹터널 애니멀스,8,깜놀했다....흡입력있는 스릴러 복수극
341965,드라마,빅쇼트,10,"어그러진 법, 달아나버린 금융인들의 도덕성의 추악함을 잘 드러냈고 몰입도 훌륭하고 ..."
115510,드라마,언포기버블,9,마지막 장면...눈물이 흘렀다
442965,드라마,이퀄스,9,어릴적 연애감정을 소비하고 싶지 않았다~ 그후 30대가 되니까혼자 연애스킬도없는 감...
777922,드라마,발신제한,2,"발신제한이 아니라 관람제한으로 제목을 바꿔야할듯...,우진씨 연기말고는 어설픈 세트..."
69548,드라마,오베라는 남자,8,이 영화의 마지막 씬이모든걸 말해 주었다오베가 무엇을 지키고 싶었는지


In [7]:
# 결측치 확인
# => 1202개(판타지)
# => 11069(드라마)
df.isna().sum()

genre             0
movie_name        0
score             0
review        11069
dtype: int64

In [8]:
# 결측치 리뷰 확인
df[df['review'].isna()].head(10)

,genre,movie_name,score,review
443,드라마,그린 북,10,NaN
1399,드라마,가버나움,10,NaN
1962,드라마,원더,10,NaN
1986,드라마,원더,10,NaN
2008,드라마,원더,10,NaN
2011,드라마,원더,10,NaN
2013,드라마,원더,10,NaN
2042,드라마,원더,10,NaN
2119,드라마,원더,10,NaN
2120,드라마,원더,8,NaN


In [9]:
df[df['review'].isna()].sample(10)

,genre,movie_name,score,review
226863,드라마,다크 워터스,9,NaN
783054,드라마,눈물이 주룩주룩,10,NaN
58076,드라마,"미안해요, 리키",9,NaN
10927,드라마,헬프,10,NaN
202981,드라마,1급기밀,10,NaN
41147,드라마,시월애,10,NaN
522725,드라마,위핏,10,NaN
269943,드라마,보이후드,10,NaN
302816,드라마,디 아워스,10,NaN
353676,드라마,블루 재스민,10,NaN


In [10]:
# 결측치 제거
df = df.dropna()

In [11]:
# 중복되는 리뷰 확인
# => 10519개(판타지)
# => 151014개(드라마)
duplication_check = df.duplicated(['review'])
duplication_check.sum()

151014

In [12]:
# 중복데이터 확인
dup = df[duplication_check == 1]
dup.sample(10)

,genre,movie_name,score,review
321886,드라마,웨이 백,10,마음에 듭니다.
792102,드라마,엑스 파일: 나는 믿고 싶다,3,답답~~~하다. 'X파일'의 명성에 먹칠을 한 영화. 딱3점이 알맞음.
262725,드라마,본 투 비 블루,10,관람객
146364,드라마,툴리,9,관람객
1710,드라마,가버나움,8,관람객
310537,드라마,프란츠,10,관람객
763921,드라마,싱크홀,5,관람객
243484,드라마,달라스 바이어스 클럽,10,관람객
463067,드라마,빅 아이즈,9,관람객
474692,드라마,클라우즈 오브 실스마리아,8,관람객


In [13]:
# 유니크한 중복데이터 확인
print(len(dup['review'].unique()))
print(dup['review'].unique()[:10])

5826
['관람객' '스포일러가 포함된 감상평입니다. 감상평 보기' '많은 사람들이 꼭 봤으면 하는 영화'
 '"힘겨운 싸움을 하는 모든 이들에게 친절하라."' '마음이 따뜻해지는 영화' 'I miss you'
 '가상의 히어로가 아니라, 진짜 인도의 영웅 이야기. 영화 런타임 긴데 지루하지 않고 재밌습니다' '당갈당갈~ 당갈당갈~'
 '진짜 쩐다... 내생의 최고의 영화엿음..' '안 보면 아쉬운 영화 꼭 한 번쯤 봤으면 좋겠다']


In [14]:
# 중복 제거
# 데이터 76553개(판타지), 654076(드라마)
df = df.drop_duplicates(['review'])
len(df)

654076

----
# **2. Data Preprocessing**
----
 - 리뷰에서 키워드가 될 수 있는 **명사만 추출**
 - 형태소 분석기(**Okt**, **Mecab**) 비교
 - **Mecab** 사용
    - 매우 빠른 실행속도 (65만개 51분 43초(Okt) VS 1분 23초(Mecab))
    - 더 세부적인 품사태깅 (일반명사, 고유명사, 의존명사, 수사 등)
 - Mecab으로 **일반명사(NNG)**, **고유명사(NNP)**만 추출
 - 명사추출 후 **불용어 제거**, **1글자 단어** 제거
 

## 분석 성능 비교

### **1) 분석 품질**

In [18]:
# 형태소 분석기 이용해 명사만 뽑아내는 과정
# 형태소 분석기는 Mecab 사용 (okt, kkma, Komoran 등등 존재)
from konlpy.tag import Okt, Mecab

okt = Okt()
mecab = Mecab()

In [ ]:
# 134, 64
okt.pos(df['review'][64])

[('최근', 'Noun'),
 ('에', 'Josa'),
 ('본', 'Verb'),
 ('영화', 'Noun'),
 ('중', 'Noun'),
 ('최고다', 'Noun'),
 ('.', 'Punctuation'),
 ('난', 'Noun'),
 ('이런', 'Adjective'),
 ('영화', 'Noun'),
 ('가', 'Josa'),
 ('좋더라', 'Adjective'),
 ('.', 'Punctuation'),
 ('특히', 'Adverb'),
 ('둘', 'Noun'),
 ('의', 'Josa'),
 ('우정은', 'Noun'),
 ('빛났다', 'Verb'),
 ('.', 'Punctuation')]

In [ ]:
# NNG:일반명사, NNP: 고유명사, NNB: 의존명사, NR: 수사, MAG: 일반부사
mecab.pos(df['review'][64])

[('최근', 'NNG'),
 ('에', 'JKB'),
 ('본', 'VV+ETM'),
 ('영화', 'NNG'),
 ('중', 'NNB'),
 ('최고', 'NNG'),
 ('다', 'VCP+EF'),
 ('.', 'SF'),
 ('난', 'NP+JX'),
 ('이런', 'MM'),
 ('영화', 'NNG'),
 ('가', 'JKS'),
 ('좋', 'VA'),
 ('더라', 'EF'),
 ('.', 'SF'),
 ('특히', 'MAG'),
 ('둘', 'NR'),
 ('의', 'JKG'),
 ('우정', 'NNG'),
 ('은', 'JX'),
 ('빛났', 'VV+EP'),
 ('다', 'EF'),
 ('.', 'SF')]

#### **Okt normalizing 테스트**

In [ ]:
# 문장 정규화 테스트
# okt.normalize 사용
review_list = df['review'].tolist()
n_review = []
for review in tqdm(review_list):
    result = okt.normalize(review)
    n_review.append(result)
n_review[:7]

  0%|          | 0/654076 [00:00<?, ?it/s]

['폭설내리는밤 경찰이 두번째로 주인공이 몰고가는 차를 세웠을때 우린 경찰의 대사나 행동을 예측했을 것이다. 하지만 그 예측 자체가 영화보는내내 우리에게 생긴 편견임을 알고 뒤통수를 맞은 느낌이었다!',
 "우리가 말모이와 스윙키즈처럼 중요한 역사적 소재에 빚지고도 허구의 신파 한가득 버무려놓아 눈물 쥐어짠 걸 의미있는 영화라 자아도취해 있을 동안, 헐리우드는 실화 소재로 이런 '작품'을 배출하는 경지에 ...",
 '하.. 이런 영화가 상영관수, 상영시간을 많이 늘려야 되는데.. 진짜 예고편만 화려한 거품영화만 너무 독차지 하고있으니..아쉽',
 '보고나면 KFC 치킨이 존나 떙기는 영화임..',
 '빗속 셜리의 대사는 소름이 끼쳤다',
 '정말 많은 여운이 남은 영화네요. 꼭 봐야 할 영화입니다',
 '비고 모텐슨 형님 반지의 제왕에서 꽃미남 왕으로 열연하던게 엊그제 같은데 중년 마초역도 멋있잖아..']

In [ ]:
# 정규화 전, 후 차이 확인
# 이건 => 이것은
# 재밋엇어 => 재밌엇어
# 반복되는 문자 처리
# 큰차이 X

cnt = 0
for n, (i, j) in enumerate(zip(review_list, n_review)):
    if n == 1000:
        break
    if i != j:
        cnt += 1
        print(i)
        print(j)
print(cnt)

인생영화 하나 나왔네요!! 강추합니당
인생영화 하나 나왔네요!! 강추합니다
두 남자의 연기, 밀도 있는 스토리, 올드팝과 재즈, 중간중간 위트까지.. 크리스마스때 봤음 더 좋을 영화! 최고임다!
두 남자의 연기, 밀도 있는 스토리, 올드팝과 재즈, 중간중간 위트까지.. 크리스마스때 봤음 더 좋을 영화! 최고입니다!
댓글 단 모든분들께 감사드립니다당신들 덕분에 이 영화를 보았고 영화 덕분에삶의 진정한 가치가 뭔지 배웁니다
댓글 단 모든분들께 감사드립니다당신들 덕분에 이 영화를 보았고 영화 덕분에삶의 진정한 가치가 뭐인지 배웁니다
한국영화에 밀려 심야영화로 간신히 봄 도대체 뭥미
한국영화에 밀려 심야영화로 간신히 봄 도대체 뭐임
오늘보고왔는뎁 넘좋았어요 보고오길잘했다는 생각이드네요 실화였다니이이이 아정말재미이어고 좋아어요 꼭보세용ㅎㅎ^^대박나세용^^♥♥
오늘보고왔는뎁 넘좋았어요 보고오길잘했다는 생각이드네요 실화였다니이이이 아정말재미이어고 좋아어요 꼭보세요ㅎㅎ^^대박나세요^^♥♥
진짜 정말 멋잇다 이건 또 보고싶다
진짜 정말 멋잇다 이것은 또 보고싶다
마지막 나의 입을 ^ㅡㅡㅡㅡ ^ 이렇게 만들어주고 끝났다
마지막 나의 입을 ^ㅡㅡㅡ ^ 이렇게 만들어주고 끝났다
피아노 치는거 합성임? 진짜 치는거임?난 웨슬리 스나입스 형님인줄 알았음.이게 작품상받고 왜논란 됐나몰라.로마 보다 백배 재밋음
피아노 치는거 합성임? 진짜 치는거임?난 웨슬리 스나입스 형님인줄 알았음.이게 작품상받고 왜논란 됐나몰라.로마 보다 백배 재밌음
정말 재밋네요 왠만하면 귀찬아서 평점 안쓰는데소소하게 재밌기도 하면서 따듯하고...내용에 교훈도 많으면서마치 잠깐 어디 여행다녀온 느낌입니다제가 본영화중에 손에 꼽아요...담에 또봐야지...그린북 최고
정말 재밌네요 왠만하면 귀찬아서 평점 안쓰는데소소하게 재밌기도 하면서 따듯하고...내용에 교훈도 많으면서마치 잠깐 어디 여행다녀온 느낌입니다제가 본영화중에 손에 꼽아요...담에 또봐야지...그린북 최고
정말 이런 영화가 많이 떠야하는 건데... 보는 내내 

### **2) 소요시간**

In [ ]:
# 51분 43초
okt_nouns = df['review'].apply(okt.nouns)
okt_nouns[:10]

In [19]:
# 1분 23초
# Mecab으로 명사 추출

# 문장을 입력받아 품사태깅 후 NNG와 NNP만 추출하는 함수
def NNGNNP(sentence):
    nouns = []
    result = mecab.pos(sentence)
    for element in result:
        if element[1] == 'NNG' or element[1] == 'NNP':
            nouns.append(element[0])
    return nouns


# NNG, NNP 명사만 추출한 Review라는 새로운 col 생성
df['Review'] = ''
df['Review'] = df['review'].apply(NNGNNP)
df.head()

,genre,movie_name,score,review,Review
0,드라마,그린 북,10,폭설내리는밤 경찰이 두번째로 주인공이 몰고가는 차를 세웠을때 우린 경찰의 대사나 행...,"[폭설, 밤, 경찰, 주인공, 차, 때, 경찰, 대사, 행동, 예측, 예측, 자체,..."
1,드라마,그린 북,10,우리가 말모이와 스윙키즈처럼 중요한 역사적 소재에 빚지고도 허구의 신파 한가득 버무...,"[말모이, 스윙, 키즈, 중요, 역사, 소재, 허구, 신파, 눈물, 의미, 영화, ..."
2,드라마,그린 북,10,"하.. 이런 영화가 상영관수, 상영시간을 많이 늘려야 되는데.. 진짜 예고편만 화려...","[영화, 상영관, 수, 상영, 시간, 예고편, 거품, 영화, 독차지]"
3,드라마,그린 북,10,보고나면 KFC 치킨이 존나 떙기는 영화임..,"[치킨, 영화]"
4,드라마,그린 북,10,빗속 셜리의 대사는 소름이 끼쳤다,"[빗속, 셜리, 대사, 소름]"


In [ ]:
from time import time

In [ ]:
start = time()
okt_nouns = df['review'].apply(okt.nouns)
end = time()

print(f"{end - start:.5f} sec")

2830.79908 sec


In [ ]:
start = time()
mecab_nouns = df['review'].apply(NNGNNP)
end = time()

print(f"{end - start:.5f} sec")

80.46242 sec


In [ ]:
df[df['Review'].apply(len) == 1]

,genre,movie_name,score,review,Review
10,드라마,그린 북,10,관람객,[관람객]
45,드라마,그린 북,10,웰메이드(well-made)란 이런 것.,[웰메이드]
53,드라마,그린 북,10,재밌습니다. 많은 생각을 할 수 있어 더 좋습니다.,[생각]
69,드라마,그린 북,10,말모이보다 훨씬 났습니다. 이거 보세요 제발.,[말모이]
185,드라마,그린 북,10,상영관이 너무적어요ㅠㅠ,[상영관]
...,...,...,...,...,...
816121,드라마,최미역행,1,여러분 별 두번누르면 1점되요 모르는분들,[별]
816131,드라마,최미역행,1,진짜 진짜 재미 없을꺼같아 ㅡㅡ,[재미]
816138,드라마,최미역행,1,제정신이 아니군????,[제정신]
816143,드라마,최미역행,1,평정 0점이 없어 1점이라도 드립니다.,[평정]


## **불용어 제거**
 - **불용어 사전** 등록 단어 제거
    - 자주 등장하는, 상위 빈도수의 영화와 관련된 보편적인 단어
    - 부사, 형용사적 의미를 갖는 명사
    - 나라이름
 - **길이 1이하 단어** 리뷰 제거
 - **감성 labeling**

In [20]:
# 불용어 사전
# 1. 자주 등장하는, 상위 빈도수의 영화와 관련된 보편적인 단어: 영화제목들, 영화, 관람객, 생각, 평점, 하나, 작품, 아이 등
# 2. 부사, 형용사적 의미를 갖는 명사: 정말, 진짜, 최고, 그냥 등등
# 3. 국가명

# 불용어 사전 정의
stopword = ['정말', '진짜', '최고', '그냥', '마지막', '다시', '완전', '정도', '역시', '처음', '지금', '이건', '별로', '때문', '뭔가', '가장', '조금', '제일',
            '영화', '관람객', '생각', '평점', '하나', '작품', '아이', '최악', '일본', '미국', '중국', '호주', '프랑스', '한국', '인도']

# 영화 타이틀 추가
for title in df['movie_name'].unique().tolist():
    stopword.append(title)

# 불용어 사전 확인
print(len(stopword))
print(stopword)

1148
['정말', '진짜', '최고', '그냥', '마지막', '다시', '완전', '정도', '역시', '처음', '지금', '이건', '별로', '때문', '뭔가', '가장', '조금', '제일', '영화', '관람객', '생각', '평점', '하나', '작품', '아이', '최악', '일본', '미국', '중국', '호주', '프랑스', '한국', '인도', '그린 북', '가버나움', '원더', '아일라', '먼 훗날 우리', '당갈', '쇼생크 탈출', '덕구', '보헤미안 랩소디', '포레스트 검프', '위대한 쇼맨', '인생은 아름다워', '헬프', '캐스트 어웨이', '히든 피겨스', '쉰들러 리스트', '집으로...', '죽은 시인의 사회', '아이 캔 스피크', '동주', '여인의 향기', '미스 슬로운', '굿바이 마이 프랜드', '굿 윌 헌팅', '지상의 별처럼', '아마데우스', '자산어보', '빌리 엘리어트', '두 교황', '그대를 사랑합니다', '아이 엠 샘', '피아니스트', '시네마 천국', '달링', '소원', '블라인드 사이드', '패왕별희 디 오리지널', '뷰티풀 투모로우', '우리들의 일그러진 영웅', '코러스', '8월의 크리스마스', '일급 살인', '허스토리', '내 이름은 칸', '록키', '나, 다니엘 블레이크', '시월애', '언플랜드', '인생', '우리들', '공동경비구역 JSA', '천국의 아이들', '맨발의 꿈', '랜드 오브 마인', '세상에서 가장 아름다운 이별', '바람', '도가니', '온리 더 브레이브', '또 하나의 약속', '피아니스트의 전설', '신데렐라 맨', '투게더', '기적', '뷰티풀 마인드', '광해, 왕이 된 남자', '쿨 러닝', '친정엄마', '미안해요, 리키', '집으로 가는 길', '바람과 함께 사라지다', '타인의 삶', '라이언', '말아톤', '킹콩을 들다', '맨 오브 오너', '파워 오브 원', '1987', '핵소 고지', '벤허', '로미오와 줄리엣', 

In [21]:
# 불용어 제거 함수
def RemoveStopWord(ls):
    clean_review = []
    for element in ls:
        if len(element) == 1 or element in stopword:
            continue
        clean_review.append(element)
    return clean_review

df['Review'] = df['Review'].apply(RemoveStopWord)
df.head()

,genre,movie_name,score,review,Review
0,드라마,그린 북,10,폭설내리는밤 경찰이 두번째로 주인공이 몰고가는 차를 세웠을때 우린 경찰의 대사나 행...,"[폭설, 경찰, 주인공, 경찰, 대사, 행동, 예측, 예측, 자체, 편견, 뒤통수,..."
1,드라마,그린 북,10,우리가 말모이와 스윙키즈처럼 중요한 역사적 소재에 빚지고도 허구의 신파 한가득 버무...,"[스윙, 키즈, 중요, 역사, 소재, 허구, 신파, 눈물, 의미, 자아, 동안, 헐..."
2,드라마,그린 북,10,"하.. 이런 영화가 상영관수, 상영시간을 많이 늘려야 되는데.. 진짜 예고편만 화려...","[상영관, 상영, 시간, 예고편, 거품, 독차지]"
3,드라마,그린 북,10,보고나면 KFC 치킨이 존나 떙기는 영화임..,[치킨]
4,드라마,그린 북,10,빗속 셜리의 대사는 소름이 끼쳤다,"[빗속, 셜리, 대사, 소름]"


In [22]:
# 전처리 후 길이가 1이하 리뷰 제거
review_len = 2
df = df[df['Review'].apply(len) >= review_len]
df.head()

,genre,movie_name,score,review,Review
0,드라마,그린 북,10,폭설내리는밤 경찰이 두번째로 주인공이 몰고가는 차를 세웠을때 우린 경찰의 대사나 행...,"[폭설, 경찰, 주인공, 경찰, 대사, 행동, 예측, 예측, 자체, 편견, 뒤통수,..."
1,드라마,그린 북,10,우리가 말모이와 스윙키즈처럼 중요한 역사적 소재에 빚지고도 허구의 신파 한가득 버무...,"[스윙, 키즈, 중요, 역사, 소재, 허구, 신파, 눈물, 의미, 자아, 동안, 헐..."
2,드라마,그린 북,10,"하.. 이런 영화가 상영관수, 상영시간을 많이 늘려야 되는데.. 진짜 예고편만 화려...","[상영관, 상영, 시간, 예고편, 거품, 독차지]"
4,드라마,그린 북,10,빗속 셜리의 대사는 소름이 끼쳤다,"[빗속, 셜리, 대사, 소름]"
6,드라마,그린 북,10,비고 모텐슨 형님 반지의 제왕에서 꽃미남 왕으로 열연하던게 엊그제 같은데 중년 마초...,"[비고, 모텐슨, 반지, 제왕, 꽃미남, 열연, 중년, 마초]"


In [ ]:
# 감성 Labeling (평점 8점 이상 긍정, 3점 이하 부정)
positive_score = 8
negative_score = 3
df['sentiment'] = np.where(df.score >= positive_score, 1,
                           np.where(df.score <= negative_score, 0, 100))
df.head()

,genre,movie_name,score,review,Review,sentiment
0,드라마,그린 북,10,폭설내리는밤 경찰이 두번째로 주인공이 몰고가는 차를 세웠을때 우린 경찰의 대사나 행...,"[폭설, 경찰, 주인공, 경찰, 대사, 행동, 예측, 예측, 자체, 편견, 뒤통수,...",1
1,드라마,그린 북,10,우리가 말모이와 스윙키즈처럼 중요한 역사적 소재에 빚지고도 허구의 신파 한가득 버무...,"[스윙, 키즈, 중요, 역사, 소재, 허구, 신파, 눈물, 의미, 자아, 동안, 헐...",1
2,드라마,그린 북,10,"하.. 이런 영화가 상영관수, 상영시간을 많이 늘려야 되는데.. 진짜 예고편만 화려...","[상영관, 상영, 시간, 예고편, 거품, 독차지]",1
4,드라마,그린 북,10,빗속 셜리의 대사는 소름이 끼쳤다,"[빗속, 셜리, 대사, 소름]",1
6,드라마,그린 북,10,비고 모텐슨 형님 반지의 제왕에서 꽃미남 왕으로 열연하던게 엊그제 같은데 중년 마초...,"[비고, 모텐슨, 반지, 제왕, 꽃미남, 열연, 중년, 마초]",1


In [ ]:
# 인덱스 초기화
df = df.reset_index()
df = df.drop(['index'], axis=1)

In [ ]:
df.head(10)

,genre,movie_name,score,review,Review,sentiment
0,드라마,그린 북,10,폭설내리는밤 경찰이 두번째로 주인공이 몰고가는 차를 세웠을때 우린 경찰의 대사나 행...,"[폭설, 경찰, 주인공, 경찰, 대사, 행동, 예측, 예측, 자체, 편견, 뒤통수,...",1
1,드라마,그린 북,10,우리가 말모이와 스윙키즈처럼 중요한 역사적 소재에 빚지고도 허구의 신파 한가득 버무...,"[스윙, 키즈, 중요, 역사, 소재, 허구, 신파, 눈물, 의미, 자아, 동안, 헐...",1
2,드라마,그린 북,10,"하.. 이런 영화가 상영관수, 상영시간을 많이 늘려야 되는데.. 진짜 예고편만 화려...","[상영관, 상영, 시간, 예고편, 거품, 독차지]",1
3,드라마,그린 북,10,빗속 셜리의 대사는 소름이 끼쳤다,"[빗속, 셜리, 대사, 소름]",1
4,드라마,그린 북,10,비고 모텐슨 형님 반지의 제왕에서 꽃미남 왕으로 열연하던게 엊그제 같은데 중년 마초...,"[비고, 모텐슨, 반지, 제왕, 꽃미남, 열연, 중년, 마초]",1
5,드라마,그린 북,10,"2019년 아카데미 작품상은 그린 북, 남우조연상은 마허샬라 알리!","[아카데미, 작품상, 남우, 조연상, 마허, 샬라, 알리]",1
6,드라마,그린 북,10,경찰관에게 메리크리스마스 하는 장면..너무 조하 따뜻해..,"[경찰관, 메리, 크리스마스, 장면, 조하]",1
7,드라마,그린 북,10,'우정' 과 '인종 차별' 모두를 아우르는 뛰어난 소재에 뒷받침하는 두 배우들의 뛰...,"[우정, 인종, 차별, 모두, 소재, 뒷받침, 배우, 심리, 변화, 연기, 디테일,...",1
8,드라마,그린 북,10,재미있고 유머러스한 영화. 관람 후에 마음에 뭔가 좋은게 생기는 기분,"[관람, 마음, 기분]",1
9,드라마,그린 북,10,마지막 부분에 경찰관을 의심했던 나마저도 편견을 가지고 있었다..나부터 편견을 버려...,"[부분, 경찰관, 의심, 편견, 편견, 사실]",1


----
# **3. Save**
----

In [ ]:
# pickle로 data를 list 형태로 저장 (pickle을 이용하면 데이터 타입도 유지하면서 저장가능, Review:list타입)
import pickle

PATH = '/content/drive/MyDrive/final-project/processed_data/data/'
file_name = f'preprocessing_{genre}_0513.pkl'
with open(PATH + file_name, 'wb') as f:
    pickle.dump(df, f, protocol=4)

### Save Test

In [ ]:
# pickle로 저장한 데이터 불러오기
import pickle

genre = '판타지'

with open(PATH + f'/movie_{genre}_0513.pkl', 'rb') as f:
    data = pickle.load(f)

data.head()

,genre,movie_name,score,review,Review
0,판타지,반지의 제왕: 왕의 귀환,10,내가 극장에서 이 영화를 봣다는게 너무 자랑스럽고 감동적입니다,"[극장, 자랑, 감동]"
1,판타지,반지의 제왕: 왕의 귀환,10,현존하는 최고의 판타지영화... 퀄리티가 절대 애들용이 아님,"[현존, 판타지, 퀄리티]"
2,판타지,반지의 제왕: 왕의 귀환,10,왜 이영화를이제서야봤는지ㅠㅠ 일편부터연달아보는데 재밌다가도 시간가는게너무안타까웠음ㅠ...,"[시간, 반지, 제왕, 명작, 가슴, 며칠, 행복]"
3,판타지,반지의 제왕: 왕의 귀환,10,몇년이 지난 영화인데도 소름돋는영화.,[소름]
4,판타지,반지의 제왕: 왕의 귀환,10,반지의제왕 안본사람은 있어도한번본 사람은 없다는게 사실인가요,"[반지, 제왕, 사람, 사람, 사실]"


In [ ]:
# Review 데이터 타입 list 확인
print(data['Review'][0])
type(data['Review'][0])

['극장', '자랑', '감동']


list